<a href="https://colab.research.google.com/github/jorgelum/EQ/blob/main/Destila%C3%A7%C3%A3o_flash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Unidade Curricular**: Operações Unitárias 3 \
**Coordenador**: Profº Dr. Alexandre Argondizo \
**Autor**: Jorge Luiz Mendes

## Bibliotecas

In [1]:
import pandas as pd #Tratamento de dados
import numpy as np
import scipy.optimize as opt #Otimização
import matplotlib.pyplot as plt # Gráficos

## Informações Gerais (exercícios 1 e 2)

Equação de equlíbrio para o sistema de hidrocarbonetos leves:

\begin{equation}
    \ ln \ K =  - \frac{A}{T^2} + B \ - C \ ln(P) + \frac{D}{P} + \frac{E}{P^2}
\end{equation}
*Sendo P(psia) e T(°R)

Cálculo de composição do flash:
\begin{equation}
  x_i = \frac{z_i}{1 + (K_i - 1)\frac{V}{F}}
\end{equation}


\begin{equation}
  y_i = \frac{z_i K_i}{1 + (K_i - 1)\frac{V}{F}}
\end{equation}

\begin{equation}
  \sum_{i = 1}^{C}{x_i}  = 1
\end{equation}

\begin{equation}
  \sum_{i = 1}^{C}{y_i}  = 1
\end{equation}

In [2]:
#Coeficientes A, B, C, D e E
dados = {'Etileno':[600076.9,7.90595,0.84677,0,42.94594],
         'Etano':[687248.2,7.90694,0.866,0,49.02654],
         'Propano':[970688.6,7.15059,0.76984,6.90224,0],
         'Butano':[1280557,7.94986,0.06455,0,0]}
pd.DataFrame(data = dados,index = ['A','B','C','D','E']).T

,A,B,C,D,E
Etileno,600076.9,7.90595,0.84677,0.00000,42.94594
Etano,687248.2,7.90694,0.86600,0.00000,49.02654
Propano,970688.6,7.15059,0.76984,6.90224,0.00000
Butano,1280557.0,7.94986,0.06455,0.00000,0.00000


## Métodos para o cálculo do flash

In [3]:
#Cálculo de Equilíbrio
def EQK(P,T,dados:dict):
  n = len(dados)
  k = np.zeros(n)
  for i,j in enumerate(dados):
     A,B,C,D,E = dados[j]
     k[i] = np.exp(- A/(T**2) + B - C*np.log(P) + D/P + E/(P**2))

  return k

#Cálculo do Tanque Flash Multicomponente
def flash(mistura:list,dados:dict,P,T,f):
  n = len(mistura)
  x = np.zeros(n)
  y = np.zeros(n)
  k = np.zeros(n)

  for i in range(0,n):
     k[i] = EQK(P,T,dados)[i]
     x[i] = mistura[i]/((1+(k[i]-1)*f))
     y[i] = x[i]*k[i]

  return x,y

## Exercício 1

A mistura especificada a seguir é alimentada a um separador flash que opera a uma pressão de 200 psia.
Composição:
 - Etileno($C_2 H_4$) = 20 %
 - Etano($C_2 H_6$) = 20 %
 - Propano($C_3 H_8$) = 40 %
 - n-Butano($C_4 H_{10}$) = 20 %


 Determine a temperatura e a composição das correntes de vapor e líquido que deixam o separador se 40 % da mistura (feed) é vaporizada.

Como não havia informações a respeito de $T_{bolha}$ e $T_{orvalho}$ a primeira estimativa foi feita com base na menor T de ebulição dos componentes (T = - 89 °C / 331 °R). \
Lembrando:    
\begin{equation}
  °R = (°C + 273,15) \frac{9}{5}
\end{equation}  

In [4]:
mistura = [0.2,0.2,0.4,0.2] # Composição da alimentação em fração

f = 0.4 #fração de vaporização V/F
P = 200 #Pressão em psia
T = 331 #Temperatura em °R

print(f'Componentes - Temperatura {T}°R no flash à {P} psia')
for i,j in enumerate(dados):
  xi = flash(mistura,dados,P,T,f)[0][i]*100
  yi = flash(mistura,dados,P,T,f)[1][i]*100
  print(f'{j}: fase líquida {round(xi,2)} % | fase gasosa {round(yi,2)} %')


Componentes - Temperatura 331°R no flash à 200 psia
Etileno: fase líquida 30.71 % | fase gasosa 3.93 %
Etano: fase líquida 32.21 % | fase gasosa 1.68 %
Propano: fase líquida 66.53 % | fase gasosa 0.21 %
Butano: fase líquida 32.96 % | fase gasosa 0.56 %


Métodos de otimização para achar a temperatura do flash.

In [5]:
def FOtimizada(T): #função de otimização de 1 parâmetro
  x = sum(flash(mistura,dados,P,T,f)[0])
  y = sum(flash(mistura,dados,P,T,f)[1])
  return (x - 1)**2 + (y - 1)**2 #Forçando os somatórios a darem 1

R1 = opt.minimize_scalar(FOtimizada,bounds=(300,1000)).x
print(f'Temperatura otimizada {round(R1,2)}°R')

Temperatura otimizada 460.95°R


Utilizando a equação de Rachford-Rice.

\begin{equation}
  \sum_{i = 0}^{C}{\frac{Z_i (1- K_i)}{1 + ψ(K_i -1)}} = 0
\end{equation}

In [6]:
def RR(T,P,dados,mistura,f):
  n = len(dados)
  k = np.zeros(n)
  c = np.zeros(n)
  for i in range(0,n):
    k[i] = EQK(P,T,dados)[i]
    c[i] = mistura[i]*(1-k[i])/(1+f*(k[i] - 1))

  return (sum(c))**2

R2 = opt.minimize_scalar(RR,bounds=(300,1000),args=(P,dados,mistura,f)).x
print(f'Temperatura otimizada utilizando Rachford-Rice {round(R2,2)}°R')

Temperatura otimizada utilizando Rachford-Rice 460.95°R


In [7]:
T = 460.95 #°R
print(f'Componentes - Temperatura {T}°R no flash à {P} psia')
for i,j in enumerate(dados):
  xi = flash(mistura,dados,P,T,f)[0][i]*100
  yi = flash(mistura,dados,P,T,f)[1][i]*100
  print(f'{j}: fase líquida {round(xi,2)} % | fase gasosa {round(yi,2)} %')

Componentes - Temperatura 460.95°R no flash à 200 psia
Etileno: fase líquida 15.08 % | fase gasosa 27.38 %
Etano: fase líquida 19.31 % | fase gasosa 21.03 %
Propano: fase líquida 57.75 % | fase gasosa 13.38 %
Butano: fase líquida 7.86 % | fase gasosa 38.21 %


## Exercício 2

A mistura especificada a seguir é alimentada a um separador flash que opera a uma pressão de 200 psia.
Composição:
 - Etileno($C_2 H_4$) = 20 %
 - Etano($C_2 H_6$) = 20 %
 - Propano($C_3 H_8$) = 40 %
 - n-Butano($C_4 H_{10}$) = 20 %


 Determine a fração da alimentação que é vaporizada e as composições das correntes de vapor e líquido formadas para 3 condições:

 $T_{drum}$ = 400 ºR; 600 ºR; 525°R.

In [8]:
mistura = [0.2,0.2,0.4,0.2]

T = 400.0
#T = 600.0
#T = 525.0
P = 200

def FOtimizada(f): #função de otimização de 1 parâmetro
  x = sum(flash(mistura,dados,P,T,f)[0])
  y = sum(flash(mistura,dados,P,T,f)[1])
  return (x - 1)**2 + (y - 1)**2 #Forçando os somatórios a darem 1

R1 = opt.minimize_scalar(FOtimizada,bounds=(0,1)).x
print(f'Temperatura otimizada {round(R1,2)}°R')




Temperatura otimizada 0.18°R


In [9]:
T = 400
f = 0.18
print(f'Componentes - Temperatura {T}°R no flash à {P} psia')
for i,j in enumerate(dados):
  xi = flash(mistura,dados,P,T,f)[0][i]*100
  yi = flash(mistura,dados,P,T,f)[1][i]*100
  print(f'{j}: fase líquida {round(xi,2)} % | fase gasosa {round(yi,2)} %')

Componentes - Temperatura 400°R no flash à 200 psia
Etileno: fase líquida 21.07 % | fase gasosa 15.15 %
Etano: fase líquida 22.53 % | fase gasosa 8.49 %
Propano: fase líquida 48.23 % | fase gasosa 2.5 %
Butano: fase líquida 21.25 % | fase gasosa 14.31 %
